# MotionDataset2label with ResNet

In [1]:
%install-location /notebooks/language2motion.gt/swift-install
%install-swiftpm-flags -c release
%install '.package(path: "/notebooks/language2motion.gt/code")' Batcher ModelSupport Datasets ImageClassificationModels

Installing packages:
	.package(path: "/notebooks/language2motion.gt/code")
		Batcher
		ModelSupport
		Datasets
		ImageClassificationModels
With SwiftPM flags: ['-c', 'release']
Working in: /tmp/tmpt3u_a4v0/swift-install
[1/2] Compiling Datasets ArrayUtils.swift
[2/3] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
[3/3] Linking libjupyterInstalledPackages.so
Initializing Swift...
Installation complete!


In [2]:
import Foundation
import TensorFlow
import PythonKit

import Batcher
import ModelSupport
import Datasets
import ImageClassificationModels

In [3]:
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

('inline', 'module://ipykernel.pylab.backend_inline')


# 1-channel ResNet model

In [42]:
var model = ResNet(classCount: 5, depth: .resNet18, downsamplingInFirstStage: true, channelCount: 1)

In [43]:
let optimizer = SGD(for: model, learningRate: 0.001)

# load dataset

In [41]:
let batchSize = 10
let maxSequenceLength =  300

let dataURL = URL(fileURLWithPath: "/notebooks/language2motion.gt/data/")
let serializedDatasetURL = dataURL.appendingPathComponent("motion_dataset.motion_flag.balanced.515.plist")
let labelsURL = dataURL.appendingPathComponent("labels_ds_v2.csv")

let dataset = Motion2Label(
    batchSize: batchSize, 
    serializedDatasetURL: serializedDatasetURL,
    labelsURL: labelsURL,
    tensorWidth: maxSequenceLength
)
print("dataset.training.count: \(dataset.training.count)")
print("dataset.test.count: \(dataset.test.count)")

MotionData(motionSamples: 515)
trainTensorPairs.count = 412
testTensorPairs.count = 103
dataset.training.count: 42
dataset.test.count: 11


In [44]:
print("Starting motion2label training...")

for epoch in 1...1 {
//     print("epoch \(epoch)")
    Context.local.learningPhase = .training
    var trainingLossSum: Float = 0
    var trainingBatchCount = 0
    for batch in dataset.training.sequenced() {
        print("progress \(100.0*Float(trainingBatchCount)/Float(dataset.training.count))%")
        let (tensors, labels) = (batch.first, batch.second)
        let (loss, gradients) = valueWithGradient(at: model) { model -> Tensor<Float> in
            let logits = model(tensors)
            return softmaxCrossEntropy(logits: logits, labels: labels)
        }
        trainingLossSum += loss.scalarized()
        trainingBatchCount += 1
        optimizer.update(&model, along: gradients)
    }

    Context.local.learningPhase = .inference
    var testLossSum: Float = 0
    var testBatchCount = 0
    var correctGuessCount = 0
    var totalGuessCount = 0
    for batch in dataset.test.sequenced() {
//         print("batch")
        let (tensors, labels) = (batch.first, batch.second)
        let logits = model(tensors)
        testLossSum += softmaxCrossEntropy(logits: logits, labels: labels).scalarized()
        testBatchCount += 1

        let correctPredictions = logits.argmax(squeezingAxis: 1) .== labels
        correctGuessCount = correctGuessCount
            + Int(
                Tensor<Int32>(correctPredictions).sum().scalarized())
        totalGuessCount = totalGuessCount + batchSize
    }

    let accuracy = Float(correctGuessCount) / Float(totalGuessCount)
    print(
        """
        [Epoch \(epoch)] \
        Training loss: \(trainingLossSum  / Float(trainingBatchCount)) \
        Accuracy: \(correctGuessCount)/\(totalGuessCount) (\(accuracy*100)%) \
        Loss: \(testLossSum / Float(testBatchCount))
        """
    )
}

Starting motion2label training...
progress 0.0%
progress 2.3809524%
progress 4.7619047%
progress 7.142857%
progress 9.523809%
progress 11.904762%
progress 14.285714%
progress 16.666666%
progress 19.047619%
progress 21.428572%
progress 23.809525%
progress 26.190475%
progress 28.571428%
progress 30.952381%
progress 33.333332%
progress 35.714287%
progress 38.095238%
progress 40.47619%
progress 42.857143%
progress 45.238094%
progress 47.61905%
progress 50.0%
progress 52.38095%
progress 54.761906%
progress 57.142857%
progress 59.52381%
progress 61.904762%
progress 64.28571%
progress 66.666664%
progress 69.04762%
progress 71.42857%
progress 73.809525%
progress 76.190475%
progress 78.57143%
progress 80.95238%
progress 83.333336%
progress 85.71429%
progress 88.09524%
progress 90.47619%
progress 92.85714%
progress 95.2381%
progress 97.61905%
[Epoch 1] Training loss: 1.7782195 Accuracy: 18/110 (16.363636%) Loss: 1.942091


In [45]:
let motionSamples = Array(dataset.testMotionSamples[0..<10])
let tensorPairs: Motion2Label.SourceDataSet = motionSamples.map {
    Motion2Label.getTensorPair($0, labelsDict: dataset.labelsDict, labels: dataset.labels, tensorWidth: maxSequenceLength)
}

In [46]:
let t = Tensor(stacking: tensorPairs.map { $0.first })
t.shape

▿ [10, 300, 45, 1]
  ▿ dimensions : 4 elements
    - 0 : 10
    - 1 : 300
    - 2 : 45
    - 3 : 1


In [50]:
let logits = model.extractFeatures(t)
logits.shape

▿ [10, 512]
  ▿ dimensions : 2 elements
    - 0 : 10
    - 1 : 512
